In [3]:
import numpy as np
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import os
import subprocess
import xarray as xr

import troute.nhd_io as nhd_io
import troute.nhd_network as nhd_network
import troute.nhd_network_utilities_v02 as nnu
import troute.routing.diffusive_utils as diff_utils
import troute.routing.diffusive_utils_mainstem as diff_utils_mainstem


In [4]:
# load MC simulation results, which will be used for initial and boundary conditions
#chrtout_folder = pathlib.Path("/glade/work/adamw/forcing/florence_event")
chrtout_folder = pathlib.Path("/home/APD/inland_hydraulics/florence_testcase/glade/work/jamesmcc/domains/private/florence_v21/FORCING_AnA_channel-only")
chrtout_files = sorted(chrtout_folder.glob("*CHRTOUT_DOMAIN1"))


In [5]:
with xr.open_mfdataset(
    chrtout_files, 
    combine="nested",
    concat_dim="time"
) as ds:
    
    #  lateral inflows (WRF - generated)
    q_lateral = pd.DataFrame(
        ds['qBucket'].values.T + ds['qSfcLatRunoff'].values.T,
        index=ds['feature_id'].values,
        columns=ds.time.values,
    )
    
    # wrf simulated flow (WRF - generated)
    q_wrf = pd.DataFrame(
        ds['streamflow'].values.T,
        index=ds['feature_id'].values,
        columns=ds.time.values,
    )

In [4]:
q_wrf

,2018-08-01 01:00:00,2018-08-01 02:00:00,2018-08-01 03:00:00,2018-08-01 04:00:00,2018-08-01 05:00:00,2018-08-01 06:00:00,2018-08-01 07:00:00,2018-08-01 08:00:00,2018-08-01 09:00:00,2018-08-01 10:00:00,...,2018-09-20 09:00:00,2018-09-20 10:00:00,2018-09-20 11:00:00,2018-09-20 12:00:00,2018-09-20 13:00:00,2018-09-20 14:00:00,2018-09-20 15:00:00,2018-09-20 16:00:00,2018-09-20 17:00:00,2018-09-20 18:00:00
10054208,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10960648,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10054304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10961050,1.573066,2.621095,1.475461,2.968772,1.601116,0.973967,0.549856,0.531504,0.911510,0.929662,...,0.020568,0.020440,0.020312,0.020184,0.020058,0.019932,0.019805,0.019679,0.019552,0.019425
10961054,0.228559,0.688792,1.182018,1.971274,2.026191,0.379634,0.237042,1.366901,0.257552,0.083153,...,0.002030,0.001953,0.001902,0.001865,0.001838,0.001817,0.001799,0.001784,0.001771,0.001758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6886220,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6886474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6886710,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6887136,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
# Hand-selected segments along the Cape Fear River between gages at Tar Heel and Kelly 
diffusive_domain = [8831712,8831726,8831730,8831734,8831736,8831748,8831754,8831752,8831780,8831794,8831792,8831798,8831808,8831810,8831806,
                    8831818,8831820,8831826,8831830,8831832,8831840,8831846,8831848,8831852,8831858,8834892,8834894,8834898,8834904,8834908,
                    8834912,8834918,8834924]

diffusive_domain = [8831712,8831726,8831730] # only headsegments of reaches on a mainstem.

In [7]:
#args = nhd_io.read_custom_input_new("/glade/u/home/adamw/projects/t-route/test/jobs/V03/Cape_Fear_test_MC_V3.yaml")
args = nhd_io.read_custom_input_new("/home/dongha.kim/github/t-route/test/input/yaml/Cape_Fear_test_MC_V3.yaml")
supernetwork_parameters = args[1]

In [8]:
connections, param_df, _, _ = nnu.build_connections(
        supernetwork_parameters
    )

rconn = nhd_network.reverse_network(connections)

In [9]:
# identify the tributary segments to the diffusive domain
trib_segs = []
for s in diffusive_domain:
    us_list = rconn[s]
    for u in us_list:
        if u not in diffusive_domain:
            trib_segs.append(u)

connections_trim = {k: connections[k] for k in (diffusive_domain + trib_segs)}
connections_trim[8831730]=[]  ## temporary fix to make proper TW link.
network_break_segments = set()
independent_networks, reaches_bytw, rconn = nnu.organize_independent_networks(
        connections_trim,
        network_break_segments,
    )

tw = list(nhd_network.headwaters(rconn))[0]
reach_list=reaches_bytw[tw]
rconn=independent_networks[tw]
connections_trim= nhd_network.reverse_network(rconn)
mainstem_headseg_list= diffusive_domain

In [10]:
'''
# build initial conditions array
initial_conditions = q_wrf.loc[diffusive_domain].iloc[:,0].to_numpy()
# test: q_wrf.loc[8831712].iloc[0]
# test: q_lateral.loc[8831712].iloc[:]

# mask channel geometry data
geo_cols = param_df.columns
geo_index = param_df.loc[diffusive_domain].index
geo_data = param_df.loc[diffusive_domain].to_numpy()

# lateral inflow data
qlat_data = q_lateral.loc[diffusive_domain].to_numpy()
'''

In [10]:
diffusive_trib_domain=list(set(diffusive_domain)|set(trib_segs))

#q_wrf.loc[8831712].iloc[0]
#q_lateral.loc[8831712].iloc[:]

# mask channel geometry data
param_df_sub= param_df.loc[
                        diffusive_trib_domain,
                        ["bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0", "alt"],
                    ].sort_index()
geo_cols = param_df_sub.columns.values
geo_index = param_df_sub.index.values
geo_data = param_df_sub.values

# build initial conditions array
q0 = q_wrf.loc[geo_index].iloc[:,0].to_numpy()
q0_2 = [[val] for val in q0]
initial_conditions = np.asarray(q0_2)

# lateral inflow data
qlat_data = q_lateral.loc[geo_index].to_numpy()
# MC data
q_wrf_data= q_wrf.loc[geo_index].to_numpy()

In [13]:
initial_conditions


array([[4.5101650e-08],
       [2.3536177e-07],
       [4.9835037e-08],
       [6.5052728e-07],
       [5.7535534e-08],
       [5.5667351e-07],
       [8.1289826e-08]], dtype=float32)

In [39]:
idx=0
nts_qt_g=24
frj=1
q_trib_g = np.zeros((nts_qt_g, 7))
q_trib_g[:,frj] = q_wrf_data[idx][0:nts_qt_g]
q_trib_g[:,frj]

array([4.51016504e-08, 1.90562631e-07, 4.55529886e-07, 8.40725363e-07,
       1.33883952e-06, 1.94059430e-06, 2.63723564e-06, 3.42080807e-06,
       4.28451449e-06, 5.25425367e-06, 6.33633590e-06, 7.51651669e-06,
       8.78337232e-06, 1.01274472e-05, 1.15404100e-05, 1.30146327e-05,
       1.45430777e-05, 1.61193020e-05, 1.77372804e-05, 1.93914275e-05,
       2.10771905e-05, 2.27913188e-05, 2.45315332e-05, 2.62962021e-05])

In [25]:
ts=[value for i, value in enumerate(q_wrf_data[idx]) if i<nts_qt_g] 
ts2=np.asarray(ts)

q_trib_g[:,frj]=ts2
q_trib_g[:,frj]

array([4.51016504e-08, 1.90562631e-07, 4.55529886e-07, 8.40725363e-07,
       1.33883952e-06, 1.94059430e-06, 2.63723564e-06, 3.42080807e-06,
       4.28451449e-06, 5.25425367e-06, 6.33633590e-06, 7.51651669e-06,
       8.78337232e-06, 1.01274472e-05, 1.15404100e-05, 1.30146327e-05,
       1.45430777e-05, 1.61193020e-05, 1.77372804e-05, 1.93914275e-05,
       2.10771905e-05, 2.27913188e-05, 2.45315332e-05, 2.62962021e-05])

In [76]:
ts=q_wrf.loc[8831724]
ts2=ts[0:24]
ts2

2018-08-01 01:00:00    5.753553e-08
2018-08-01 02:00:00    2.429142e-07
2018-08-01 03:00:00    5.846061e-07
2018-08-01 04:00:00    1.096352e-06
2018-08-01 05:00:00    1.788694e-06
2018-08-01 06:00:00    2.675559e-06
2018-08-01 07:00:00    3.776786e-06
2018-08-01 08:00:00    5.117603e-06
2018-08-01 09:00:00    6.768681e-06
2018-08-01 10:00:00    8.853116e-06
2018-08-01 11:00:00    1.142068e-05
2018-08-01 12:00:00    1.452589e-05
2018-08-01 13:00:00    1.822415e-05
2018-08-01 14:00:00    2.256801e-05
2018-08-01 15:00:00    2.760793e-05
2018-08-01 16:00:00    3.338756e-05
2018-08-01 17:00:00    3.994215e-05
2018-08-01 18:00:00    4.729958e-05
2018-08-01 19:00:00    5.548139e-05
2018-08-01 20:00:00    6.448665e-05
2018-08-01 21:00:00    7.408667e-05
2018-08-01 22:00:00    8.406641e-05
2018-08-01 23:00:00    9.219642e-05
2018-08-02 00:00:00    1.003316e-04
Name: 8831724, dtype: float32

In [34]:
q_wrf_data[0]

array([4.5101650e-08, 1.9056263e-07, 4.5552989e-07, ..., 1.6292465e-01,
       1.4604753e-01, 1.3417843e-01], dtype=float32)

In [84]:
segID=8831730
idx_segID = np.where(geo_index == segID)
dx = geo_data[idx_segID, 3]
print(dx)
#tlf = qlat_data[idx_segID, tsi]
tlf=q_lateral.loc[segID]
tlf2=tlf[0:24]
#print(tlf2)
tlf3= [x/dx for i, x in enumerate(tlf2)]
tlf3




[[1962.]]


[array([[1.964473e-09]], dtype=float32),
 array([[3.9865697e-09]], dtype=float32),
 array([[6.075852e-09]], dtype=float32),
 array([[8.211286e-09]], dtype=float32),
 array([[1.0378305e-08]], dtype=float32),
 array([[1.2572645e-08]], dtype=float32),
 array([[1.4801389e-08]], dtype=float32),
 array([[1.7066174e-08]], dtype=float32),
 array([[1.9360339e-08]], dtype=float32),
 array([[2.1689543e-08]], dtype=float32),
 array([[2.4055677e-08]], dtype=float32),
 array([[2.6441377e-08]], dtype=float32),
 array([[2.8825145e-08]], dtype=float32),
 array([[3.1200376e-08]], dtype=float32),
 array([[3.3562902e-08]], dtype=float32),
 array([[3.590261e-08]], dtype=float32),
 array([[3.8208977e-08]], dtype=float32),
 array([[4.048442e-08]], dtype=float32),
 array([[4.272782e-08]], dtype=float32),
 array([[4.4946084e-08]], dtype=float32),
 array([[4.714712e-08]], dtype=float32),
 array([[4.937109e-08]], dtype=float32),
 array([[5.159345e-08]], dtype=float32),
 array([[5.382164e-08]], dtype=float32)]

In [59]:
print(geo_index)
print()
param_df.loc[8831726]

[8831708 8831710 8831712 8831714 8831724 8831726 8831730]


dx       840.000000
n          0.045000
ncc        0.090000
s0         0.002080
bw        36.292484
tw        60.487476
twcc     181.462418
alt       11.890000
musk    3600.000000
musx       0.200000
cs         0.191003
Name: 8831726, dtype: float32

In [60]:
param_df_sub

,bw,tw,twcc,dx,n,ncc,cs,s0,alt
key,,,,,,,,,
8831708,2.055844,3.426407,10.279221,44.0,0.060,0.12,0.677814,0.00001,11.89
8831710,36.267906,60.446510,181.339523,2467.0,0.045,0.09,0.191060,0.00001,11.89
8831712,36.271111,60.451851,181.355545,870.0,0.045,0.09,0.191053,0.00001,11.89
8831714,4.141234,6.902058,20.706173,50.0,0.060,0.12,0.497658,0.00001,11.89
8831724,2.957945,4.929909,14.789727,40.0,0.060,0.12,0.577304,0.00001,10.14
8831726,36.292484,60.487476,181.462418,840.0,0.045,0.09,0.191003,0.00208,11.89
8831730,36.303112,60.505184,181.515549,1962.0,0.045,0.09,0.190978,0.00002,10.14


In [62]:
geo_cols

array(['bw', 'tw', 'twcc', 'dx', 'n', 'ncc', 'cs', 's0', 'alt'],
      dtype=object)

In [11]:
import yaml
custom_input_file='../../../../test/input/yaml/Florence_Benchmark_diff.yaml'
with open(custom_input_file) as custom_file:
    data = yaml.load(custom_file, Loader=yaml.SafeLoader)
diffusive_parameters = data.get("diffusive_parameters", {})  
upstream_results={}
lake_segs=[]
waterbodies_df_sub = pd.DataFrame()
waterbodies_df_sub.values
lake_segs=np.asarray(lake_segs)
wbody_params=np.array(waterbodies_df_sub.values)
qts_subdivisions=12
nsteps=288
dt=300


array([], shape=(0, 0), dtype=float64)

In [13]:
# build upstream results array
#trib_segs
#wbody_params

In [1]:
diff_inputs= diff_utils_mainstem.diffusive_mainstem_input_data_v01(
    tw,
    connections_trim,
    rconn,
    reach_list,
    mainstem_headseg_list,
    q_wrf_data,    
    diffusive_parameters,
    geo_cols,
    geo_index,
    geo_data,
    qlat_data,
    initial_conditions,
    upstream_results,
    qts_subdivisions,
    nsteps,
    dt,
    lake_segs,
    wbody_params,
)


NameError: name 'diff_utils_mainstem' is not defined